# IMDB: recursive neural networks

## Data preprocessing

### Required imports

In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import numpy as np
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


### Processing

Load the training and test data.  To limit computation time, we restrict the number of words to 5,000.

In [2]:
num_words = 5_000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)

Since the review vary in length, and we prefer to limit the computation time, we will base the classification on the first 100 features of each input sequence.

In [3]:
feature_length = 100
x_train = sequence.pad_sequences(x_train, maxlen=feature_length)
x_test = sequence.pad_sequences(x_test, maxlen=feature_length)

Now the training and test input are 2D arrays. We split the training set into a subset for actual training, and one for validation.  First we seed the random number generator to ensure reproducibility. In this case, we will use part of the 25000 test examples as valiation data.

In [4]:
np.random.seed(1234)

In [5]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train)

## GRU

### Required imports & model definition

In [6]:
from keras.layers import Activation, Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import GRU
from keras.models import Sequential
from keras.optimizers import Adam

Again, to limit training times, we restrict ourselfs to using a limited number of features.

In [7]:
vector_length = 64
num_units = 64
model = Sequential()
model.add(Embedding(num_words, vector_length, mask_zero=True,
                    input_length=feature_length))
model.add(GRU(num_units))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 64)           320000    
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                24768     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 344,833
Trainable params: 344,833
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(loss='binary_crossentropy', optimizer=Adam(),
              metrics=['accuracy'])

###    Training

In [10]:
history = model.fit(x_train, y_train, batch_size=64, epochs=10,
                    validation_data=(x_val, y_val))

Train on 18750 samples, validate on 6250 samples
Epoch 1/10
18750/18750 [==============================] - 52s 3ms/step - loss: 0.4887 - acc: 0.7486 - val_loss: 0.3762 - val_acc: 0.8371
Epoch 2/10
18750/18750 [==============================] - 52s 3ms/step - loss: 0.3100 - acc: 0.8725 - val_loss: 0.3487 - val_acc: 0.8466
Epoch 3/10
18750/18750 [==============================] - 61s 3ms/step - loss: 0.2666 - acc: 0.8947 - val_loss: 0.3572 - val_acc: 0.8418
Epoch 4/10
18750/18750 [==============================] - 54s 3ms/step - loss: 0.2284 - acc: 0.9125 - val_loss: 0.3969 - val_acc: 0.8418
Epoch 5/10
18750/18750 [==============================] - 49s 3ms/step - loss: 0.1908 - acc: 0.9288 - val_loss: 0.4025 - val_acc: 0.8440
Epoch 6/10
18750/18750 [==============================] - 55s 3ms/step - loss: 0.1577 - acc: 0.9418 - val_loss: 0.4363 - val_acc: 0.8398
Epoch 7/10
18750/18750 [==============================] - 54s 3ms/step - loss: 0.1205 - acc: 0.9583 - val_loss: 0.4833 - val_acc:

The training accuracy is much better than the validation accurcy, so the model is likely heavily overtrained.

### Testing

In [11]:
model.evaluate(x_test, y_test)

25000/25000 [==============================] - 27s 1ms/step


[0.7891314828014374, 0.8184]

## LSTM

### Required imports & model definition

In [12]:
from keras.layers.recurrent import LSTM

Again, to limit training times, we restrict ourselfs to using a limited number of features.

In [13]:
vector_length = 64
num_units = 64
model = Sequential()
model.add(Embedding(num_words, vector_length, mask_zero=True,
                    input_length=feature_length))
model.add(LSTM(num_units))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 64)           320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total params: 353,089
Trainable params: 353,089
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(loss='binary_crossentropy', optimizer=Adam(),
              metrics=['accuracy'])

###    Training

In [16]:
history = model.fit(x_train, y_train, batch_size=64, epochs=10,
                    validation_data=(x_val, y_val))

Train on 18750 samples, validate on 6250 samples
Epoch 1/10
18750/18750 [==============================] - 68s 4ms/step - loss: 0.4706 - acc: 0.7644 - val_loss: 0.3472 - val_acc: 0.8518
Epoch 2/10
18750/18750 [==============================] - 68s 4ms/step - loss: 0.3061 - acc: 0.8759 - val_loss: 0.3585 - val_acc: 0.8365
Epoch 3/10
18750/18750 [==============================] - 70s 4ms/step - loss: 0.2570 - acc: 0.8951 - val_loss: 0.3720 - val_acc: 0.8427
Epoch 4/10
18750/18750 [==============================] - 62s 3ms/step - loss: 0.2221 - acc: 0.9147 - val_loss: 0.4102 - val_acc: 0.8390
Epoch 5/10
18750/18750 [==============================] - 66s 3ms/step - loss: 0.1798 - acc: 0.9315 - val_loss: 0.4257 - val_acc: 0.8354
Epoch 6/10
18750/18750 [==============================] - 81s 4ms/step - loss: 0.1551 - acc: 0.9431 - val_loss: 0.4375 - val_acc: 0.8299
Epoch 7/10
18750/18750 [==============================] - 77s 4ms/step - loss: 0.1352 - acc: 0.9499 - val_loss: 0.5219 - val_acc:

The training accuracy is much better than the validation accurcy, so the model is likely heavily overtrained.

### Testing

In [17]:
model.evaluate(x_test, y_test)

25000/25000 [==============================] - 48s 2ms/step


[0.643139419708252, 0.8162]